# Implementando uma rede simples (MLP) usando o Keras

## Inteli - Sistemas de Informação - Programação
- **Professor**👨‍🏫: Jefferson de Oliveira Silva
- **Aluno**👨‍🎓: Pedro de Carvalho Rezende

### Objetivo🚨
Desenvolver um Perceptron utilizando Keras em Python e explicar detalhadamente cada parte do código desenvolvido. O Perceptron é um modelo de rede neural simples, mas fundamental, que serve como base para o entendimento de modelos de aprendizado mais complexos.


### Instruções📃
Escolha um dataset pronto adequado para classificação binária, evitando datasets "toy" como `Iris` ou `Pima Indians Diabetes`. Certifique-se de selecionar um dataset que ofereça desafios reais em termos de volume e complexidade.

Em seguida, explore o dataset escolhido e explique suas características principais, como o número de amostras, features, e a tarefa de classificação que ele representa.

Desenvolva um modelo sequencial em Keras com uma única camada Dense, utilizando uma unidade com a função de ativação sigmoid. Compile o modelo utilizando o otimizador adam, a função de perda binary_crossentropy, e a métrica accuracy. Inclua também a métrica F1 para uma avaliação mais completa, e explique brevemente a função de cada um desses componentes no treinamento.

Treine o modelo por 50 épocas com um batch size de 10. Após o treinamento, utilize o modelo para prever os rótulos do conjunto de teste e calcule tanto a acurácia quanto a métrica F1. Interprete os resultados, discutindo o desempenho do modelo e possíveis melhorias.

Entregue o link do caderno `.ipynb` em um repositório GitHub.

# Instalações e Importações

In [17]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import BinaryAccuracy, AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# baixando o dataset utilizado
!gdown 12FihSjn8qDfmoGjra_c4pYRFq1P6hwTg

# Exploratória do Dataset

O dataset escolhido foi Steam Store Games (https://www.kaggle.com/nikdavis/steam-store-games). 

Este dataset contém informações sobre jogos disponíveis na plataforma Steam, incluindo o nome do jogo, a descrição, o preço, a data de lançamento, a avaliação dos usuários, entre outras informações.

In [2]:
# visualização do dataset
'''
Retirei algumas colunas que não são relevantes para a análise devido os seus valores
aapid: identificador do jogo
english: se o jogo é em inglês ou não (não é relevante para a análise)
required_age: idade mínima para jogar o jogo (não é relevante para a análise)
platforms: plataformas disponíveis para o jogo (não é relevante para a análise)
steamspy_tags: tags do jogo, com generos, temas, etc (já temos o gênero)
'''
df = pd.read_csv('steam.csv', usecols= lambda col: col not in ['appid', 'english', 'required_age', 'platforms', 'steamspy_tags'])
df

,name,release_date,developer,publisher,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,Counter-Strike,2000-11-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,124534,3339,17612,317,10000000-20000000,7.19
1,Team Fortress Classic,1999-04-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,3318,633,277,62,5000000-10000000,3.99
2,Day of Defeat,2003-05-01,Valve,Valve,Multi-player;Valve Anti-Cheat enabled,Action,0,3416,398,187,34,5000000-10000000,3.99
3,Deathmatch Classic,2001-06-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,1273,267,258,184,5000000-10000000,3.99
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,Room of Pandora,2019-04-24,SHEN JIAWEI,SHEN JIAWEI,Single-player;Steam Achievements,Adventure;Casual;Indie,7,3,0,0,0,0-20000,2.09
27071,Cyber Gun,2019-04-23,Semyon Maximov,BekkerDev Studio,Single-player,Action;Adventure;Indie,0,8,1,0,0,0-20000,1.69
27072,Super Star Blast,2019-04-24,EntwicklerX,EntwicklerX,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,24,0,1,0,0,0-20000,3.99
27073,New Yankee 7: Deer Hunters,2019-04-17,Yustas Game Studio,Alawar Entertainment,Single-player;Steam Cloud,Adventure;Casual;Indie,0,2,0,0,0,0-20000,5.19


Vou modificar a coluna 'gêneros' para que apenas o primeiro gênero seja mostrado, pois sendo separado por ponto e vírgula (;) um título acumula muitos gêneros. O primeiro gênero seria o mais importante

In [5]:
split_genres = df["genres"].str.split(";", n=1, expand=True)
df["genres"] = split_genres[0]
df.head()

,name,release_date,developer,publisher,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,Counter-Strike,2000-11-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,124534,3339,17612,317,10000000-20000000,7.19
1,Team Fortress Classic,1999-04-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,3318,633,277,62,5000000-10000000,3.99
2,Day of Defeat,2003-05-01,Valve,Valve,Multi-player;Valve Anti-Cheat enabled,Action,0,3416,398,187,34,5000000-10000000,3.99
3,Deathmatch Classic,2001-06-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,1273,267,258,184,5000000-10000000,3.99
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,5250,288,624,415,5000000-10000000,3.99


In [7]:
df['owners'].value_counts()

owners
0-20000                18596
20000-50000             3059
50000-100000            1695
100000-200000           1386
200000-500000           1272
500000-1000000           513
1000000-2000000          288
2000000-5000000          193
5000000-10000000          46
10000000-20000000         21
20000000-50000000          3
50000000-100000000         2
100000000-200000000        1
Name: count, dtype: int64

In [8]:
# Mapeando a coluna 'owners' para valores numéricos de acordo com a quantidade de donos
owners_mapping = {
    '0-20000': 1,
    '20000-50000': 2,
    '50000-100000': 2,
    '100000-200000': 3,
    '200000-500000': 3,
    '500000-1000000': 4,
    '1000000-2000000': 4,
    '2000000-5000000': 4,
    '5000000-10000000': 5,
    '10000000-20000000': 5,
    '20000000-50000000': 5,
    '50000000-100000000': 5,
    '100000000-200000000': 5
}

df['owners_scaled'] = df['owners'].map(owners_mapping)

print(df[['owners', 'owners_scaled']].head())


              owners  owners_scaled
0  10000000-20000000              5
1   5000000-10000000              5
2   5000000-10000000              5
3   5000000-10000000              5
4   5000000-10000000              5


In [12]:
df = df.drop(columns=['owners'])
df

,name,release_date,developer,publisher,categories,genres,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price,popularity,owners_scaled
0,Counter-Strike,2000-11-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,124534,3339,17612,317,7.19,1,5
1,Team Fortress Classic,1999-04-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,3318,633,277,62,3.99,1,5
2,Day of Defeat,2003-05-01,Valve,Valve,Multi-player;Valve Anti-Cheat enabled,Action,0,3416,398,187,34,3.99,1,5
3,Deathmatch Classic,2001-06-01,Valve,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,0,1273,267,258,184,3.99,1,5
4,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Valve,Single-player;Multi-player;Valve Anti-Cheat en...,Action,0,5250,288,624,415,3.99,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,Room of Pandora,2019-04-24,SHEN JIAWEI,SHEN JIAWEI,Single-player;Steam Achievements,Adventure,7,3,0,0,0,2.09,1,1
27071,Cyber Gun,2019-04-23,Semyon Maximov,BekkerDev Studio,Single-player,Action,0,8,1,0,0,1.69,1,1
27072,Super Star Blast,2019-04-24,EntwicklerX,EntwicklerX,Single-player;Multi-player;Co-op;Shared/Split ...,Action,24,0,1,0,0,3.99,0,1
27073,New Yankee 7: Deer Hunters,2019-04-17,Yustas Game Studio,Alawar Entertainment,Single-player;Steam Cloud,Adventure,0,2,0,0,0,5.19,0,1


### Legenda:
- name: The name of the game.
- release_date: The release date of the game.
- developer: The developer of the game.
- publisher: The publisher of the game.
- categories: Categories the game belongs to (e.g., Single-player, Multi-player).
- genres: Genre of the game (e.g., Action).
- achievements: Number of achievements available in the game.
- positive_ratings: Number of positive ratings the game has received.
- negative_ratings: Number of negative ratings the game has received.
- average_playtime: Average playtime of the game in minutes.
- median_playtime: Median playtime of the game in minutes.
- owners: An estimate of the number of owners of the game (represented as a range).
- price: The price of the game

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27075 entries, 0 to 27074
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              27075 non-null  object 
 1   release_date      27075 non-null  object 
 2   developer         27074 non-null  object 
 3   publisher         27061 non-null  object 
 4   categories        27075 non-null  object 
 5   genres            27075 non-null  object 
 6   achievements      27075 non-null  int64  
 7   positive_ratings  27075 non-null  int64  
 8   negative_ratings  27075 non-null  int64  
 9   average_playtime  27075 non-null  int64  
 10  median_playtime   27075 non-null  int64  
 11  price             27075 non-null  float64
 12  popularity        27075 non-null  int32  
 13  owners_scaled     27075 non-null  int64  
dtypes: float64(1), int32(1), int64(6), object(6)
memory usage: 2.8+ MB


In [14]:
df.describe()

,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,price,popularity,owners_scaled
count,27075.000000,2.707500e+04,27075.000000,27075.000000,27075.00000,27075.000000,27075.000000,27075.000000
mean,45.248864,1.000559e+03,211.027147,149.804949,146.05603,6.078193,0.553241,1.492853
std,352.670281,1.898872e+04,4284.938531,1827.038141,2353.88008,7.874922,0.497166,0.836032
min,0.000000,0.000000e+00,0.000000,0.000000,0.00000,0.000000,0.000000,1.000000
25%,0.000000,6.000000e+00,2.000000,0.000000,0.00000,1.690000,0.000000,1.000000
50%,7.000000,2.400000e+01,9.000000,0.000000,0.00000,3.990000,1.000000,1.000000
75%,23.000000,1.260000e+02,42.000000,0.000000,0.00000,7.190000,1.000000,2.000000
max,9821.000000,2.644404e+06,487076.000000,190625.000000,190625.00000,421.990000,1.000000,5.000000


# Procedimentos

In [15]:
# Se a proporção de classificações positivas e negativas estiver acima de um limite, rotularemos como 1 (popular), caso contrário, 0.
threshold = 2  # Jogos com pelo menos o dobro de classificações positivas do que negativas serão considerados populares
df['popularity'] = (df['positive_ratings'] / (df['negative_ratings'] + 1)) > threshold
df['popularity'] = df['popularity'].astype(int)

# Selecionando recursos relevantes para o modelo
features = df[['achievements', 'positive_ratings', 'negative_ratings', 'average_playtime', 'owners_scaled', 'price']]
target = df['popularity'] # aqui estamos considerando a popularidade como a variável alvo, então estamos criando um target

# Dividindo o conjunto de dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, y_train.shape, X_test_scaled.shape, y_test.shape

((21660, 6), (21660,), (5415, 6), (5415,))

## Até o momento

- Então, até o momento, fizemos a importação das bibliotecas necessárias e a leitura do dataset.	
- Em seguida, fizemos uma análise exploratória do dataset, verificando as primeiras linhas, informações gerais e estatísticas descritivas.
- Também fizemos uma análise mais detalhada das colunas 'genres' e 'categories', que contêm informações sobre os gêneros e categorias dos jogos.
- Modificamos a coluna 'genres' para que apenas o primeiro gênero fosse mostrado, pois os títulos acumulavam muitos gêneros.
- Criamos, com base na proporção de classificações positivas e negativas, uma variável que se um jogo tiver pelo menos duas vezes mais classificações positivas do que negativas, ele será rotulado como “popular” (1), caso contrário, será rotulado como “não popular” (0).
- Por fim, dividimos o dataset em conjuntos de treino e teste, com 80% dos dados para treino e 20% para teste. Normalizando-os a partir do SantoScaler.


## Modelo de Rede Neural
- Agora, desenvolverei um modelo Perceptron usando Keras. O modelo terá uma única camada densa com função de ativação sigmóide para classificação binária (como foi requisitado).


In [17]:
model = Sequential([
    Dense(1, activation='sigmoid', input_shape=(X_train_scaled.shape[1],))
])

# compliando o modelo
model.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(),
    metrics=[BinaryAccuracy(name='accuracy'), AUC(name='auc')]
)

c:\Users\pedro\OneDrive\Documentos\C.1 - GitHub\M11_ponderadas\env\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


- Sequential(): Isso cria uma pilha linear de camadas. Neste caso, temos uma única camada Dense.
- Dense(1, activation='sigmoid'): Esta é a camada do Perceptron. Ela possui 1 unidade (neurônio de saída) com a função de ativação sigmoid, que é adequada para classificação binária.
- Sigmoid(): A função de ativação sigmoid é uma função de ativação comum usada em problemas de classificação binária. Ela mapeia os valores de entrada para um intervalo entre 0 e 1, o que é útil para interpretar as saídas como probabilidades.
- compile(): Esta função é usada para configurar o modelo para treinamento. 
    - Aqui, estamos usando:
        - o otimizador 'adam', que serve para otimização de gradientes; 
        - a função de perda 'binary_crossentropy', que é adequada para problemas de classificação binária;
        - a métrica 'accuracy', que é a métrica padrão para problemas de classificação;
    - Além disso, incluímos a métrica 'f1' para uma avaliação mais completa. A função de perda binary_crossentropy é adequada para problemas de classificação binária, enquanto o otimizador adam é uma boa escolha para otimização de gradientes.

## Resultados

- fit(): Este método treina o modelo por 50 épocas com um tamanho de lote (batch size) de 10. O validation_split=0.2 significa que 20% dos dados de treinamento serão usados para validação.
- As previsões são feitas no conjunto de teste, e a acurácia e o F1-score são calculados. Essas métricas dão uma indicação de quão bem o modelo está performando.

**Lembrando**:
- A acurácia é a proporção de previsões corretas em relação ao total de previsões.
- O F1-score é uma métrica que combina precisão e recall, fornecendo uma medida mais equilibrada do desempenho do modelo.

In [18]:
# treinando o modelo
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=10, validation_split=0.2, verbose=1)

y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

test_accuracy = accuracy_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

print(f'Test Accuracy: {test_accuracy}')
print(f'Test F1 Score: {test_f1}')


Epoch 1/50
1733/1733 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.5633 - auc: 0.5564 - loss: 0.6889 - val_accuracy: 0.5854 - val_auc: 0.6202 - val_loss: 0.6666
Epoch 2/50
1733/1733 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5964 - auc: 0.6311 - loss: 0.6569 - val_accuracy: 0.5903 - val_auc: 0.6259 - val_loss: 0.6606
Epoch 3/50
1733/1733 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5957 - auc: 0.6261 - loss: 0.6593 - val_accuracy: 0.5926 - val_auc: 0.6268 - val_loss: 0.6597
Epoch 4/50
1733/1733 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6037 - auc: 0.6380 - loss: 0.6572 - val_accuracy: 0.5912 - val_auc: 0.6277 - val_loss: 0.6588
Epoch 5/50
1733/1733 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6030 - auc: 0.6349 - loss: 0.6544 - val_accuracy: 0.5903 - val_auc: 0.6277 - val_loss: 0.6580
Epoch 6/50
1733/1733 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6029 - auc: 0.6340 - loss: 0.6551 - val_accuracy: 0.5916 - val_auc: 0.6291 - val_loss: 0.6574
Epoch 7/50
1733/1733 ━━━━━━━

### Conclusão

Os resultados obtidos pelo modelo Perceptron indicam uma performance moderada na tarefa de classificação binária, com, aproximadamente, uma **acurácia de 61,33%** e um **F1 Score de 0,65** no conjunto de teste.

A acurácia de 61,33% significa que o modelo foi capaz de prever corretamente aproximadamente 61% dos exemplos no conjunto de teste. Embora este valor seja superior ao acaso (considerando que se trata de uma tarefa de classificação binária), ainda há margem significativa para melhorias.

O **F1 Score de 0,65** reflete um equilíbrio razoável entre precisão (proporção de verdadeiros positivos em relação ao total de positivos preditos) e recall (proporção de verdadeiros positivos em relação ao total de positivos reais). O valor do F1 Score sugere que o modelo tem um desempenho aceitável, mas que pode ser aprimorado, especialmente em cenários onde o desbalanceamento de classes pode influenciar a performance.

Esses resultados podem ser melhorados com estratégias adicionais, como a inclusão de mais features relevantes, ajustes na arquitetura do modelo, uso de técnicas de regularização, ou mesmo a experimentação com modelos mais complexos e sofisticados. Além disso, uma análise mais profunda dos dados e uma abordagem de engenharia de features poderiam ajudar a melhorar a capacidade preditiva do modelo, resultando em métricas de performance mais elevadas.

# Melhorias do modelo e possíveis próximos passos

Aqui será listado fatores que poderia influenciar na melhoria do modelo e possíveis próximos passos para aprimorar a performance do modelo Perceptron.
- **Engenharia de Features**: Explorar e criar novas features a partir dos dados existentes pode ajudar a capturar informações mais relevantes para a tarefa de classificação.
    - Por exemplo, poderíamos criar features que representam a interação entre diferentes variáveis, ou extrair informações adicionais de colunas como 'genres' e 'categories'.
    - Além disso, técnicas como one-hot encoding, binarização de variáveis categóricas, ou mesmo a criação de features polinomiais poderiam ajudar a melhorar a capacidade preditiva do modelo.
- **Regularização**: A adição de técnicas de regularização, como L1, L2, ou elastic net, pode ajudar a evitar overfitting e melhorar a generalização do modelo.
    - A regularização penaliza os pesos do modelo, incentivando-os a permanecerem pequenos e reduzindo a complexidade do modelo.
- **Ajuste de Hiperparâmetros**: Experimentar diferentes valores para hiperparâmetros como a taxa de aprendizado, o número de épocas, o tamanho do batch, ou mesmo a arquitetura do modelo pode ajudar a encontrar uma configuração que resulte em melhor performance.
    - Técnicas como grid search, random search, ou otimização bayesiana podem ser usadas para encontrar os melhores hiperparâmetros para o modelo.
    - Por encaixe da atividade, o modelo foi treinado como designado no corpo da atividade.
- **Validação Cruzada**: Utilizar técnicas de validação cruzada, como k-fold cross-validation, pode ajudar a avaliar a capacidade de generalização do modelo e reduzir a variância das métricas de performance.
    - A validação cruzada divide o conjunto de dados em k partes, treinando o modelo em k-1 partes e avaliando-o na parte restante, repetindo o processo k vezes.
- **Modelos mais Complexos**: Experimentar modelos mais complexos, como redes neurais profundas, redes convolucionais, ou redes recorrentes, pode ajudar a capturar padrões mais sutis nos dados e melhorar a performance do modelo.
    - Modelos mais complexos podem aprender representações mais abstratas dos dados, permitindo uma melhor discriminação entre as classes.
- **Balanceamento de Classes**: Em problemas de classificação binária com classes desbalanceadas, técnicas como oversampling, undersampling, ou geração sintética de dados podem ajudar a equilibrar a distribuição das classes e melhorar a performance do modelo.
    - O balanceamento de classes pode ajudar a evitar que o modelo seja enviesado em direção à classe majoritária, resultando em métricas de performance mais realistas.